In [97]:
import jieba
import re
import itertools
import random

# 解析模式

In [62]:
def parse_pattern(pat):
    """
    解析pattern
    :param pat:
    :return:
    """
    plist = re.findall("(\?\*?\w|\w+)\s*", pat)
    ret = []
    for i in plist:
        p = {
            "type": None
        }
        if i.startswith("?"):
            if i.startswith("?*"):
                ret.append({"type": "multiple_var", "name": i[2:]})
            else:
                ret.append({"type": "single_var", "name": i[1:]})
        else:
            ret.extend([{"type": "const", "value": x} for x in jieba.lcut(i)])
    return ret

In [63]:
parse_pattern("?*x hello ?*y")

[{'type': 'multiple_var', 'name': 'x'},
 {'type': 'const', 'value': 'hello'},
 {'type': 'multiple_var', 'name': 'y'}]

# 匹配句子

In [163]:
def match_sentence(pat, saying):
    """
    匹配模式和句子
    :param pat:
    :param saying:
    :return:
    """
    patter_list = parse_pattern(pat)
    saying_list = [x for x in jieba.lcut(saying) if x.strip()]

    def find_word(w, saying_list):
        """
        查询词在list中的位置
        :param w:
        :param saying_list:
        :return:
        """
        for i, v in enumerate(saying_list):
            if v != "" and w == v:
                return i
        return -1

    # 匹配
    saying_pos = 0      # 被匹配字符串的位置
    ret_dic = {}
    for i, p in enumerate(patter_list):
        if saying_pos == len(saying_list):
            # 句子已经消耗完了，模式还没匹配完
            return False, ret_dic
        if p["type"] == "multiple_var":
            if i == len(patter_list) - 1:
                # 最后一个模式是变量， 直接返回剩余所有的即可
                ret_dic[p["name"]] = saying_list[saying_pos:]
                saying_pos = len(saying_list)
            else:
                next_pat = patter_list[i+1]
                assert next_pat['type'] == "const"      # 暂时不支持两个变量相邻
                pos = find_word(next_pat['value'], saying_list[saying_pos:])
                if pos == -1 or pos == 0:
                    # 不匹配
                    return False, ret_dic
                else:
                    ret_dic[p["name"]] = saying_list[saying_pos:pos]
                    saying_pos += pos
        elif p["type"] == "single_var":
            ret_dic[p["name"]] = saying_list[saying_pos]
            saying_pos += 1
        elif p["type"] == "const":
            if p["value"] == saying_list[saying_pos]:
                saying_pos += 1
        else:
            raise Exception("???")

    if saying_pos != len(saying_list):
        # 模式走完了，被匹配的字符串没匹配完
        return False, ret_dic
    return True, ret_dic

In [123]:
match_sentence("?X greater than ?Y", "3 greater than 2")

(True, {'X': '3', 'Y': '2'})

In [124]:
match_sentence('?*P is very good', "My dog and my cat is very good")

(True, {'P': ['My', 'dog', 'and', 'my', 'cat']})

In [125]:
match_sentence('?*x就像?*y', "小明就像小红")

(True, {'x': ['小明'], 'y': ['小红']})

In [148]:
match_sentence("'?*x你好?*y", "你好呀")

(False, {})

In [149]:
match_sentence("'?*x你好?*y", "小冰你好")

(False, {'x': ['小冰']})

# 替换句子

In [176]:
def subsitite(rule, parsed_rules):
    """
    替换句子中的变量
    :param rule:
    :param parsed_rules:
    :return:
    """
    if not rule or not parsed_rules: return []
    rule_list = parse_pattern(rule)
    ret = []
    for i in rule_list:
        a = parsed_rules.get(i.get("name"),i.get("value"))
        if isinstance(a, list):
            ret.extend(a)
        else:
            ret.append(a)
    cn_model = re.compile(u'[\u4e00-\u9fff]')
    if cn_model.search(rule):
        return "".join(ret)
    else:
        return " ".join(ret)

In [177]:
succ, pat = match_sentence("I want ?X", "I want holiday")
subsitite("What if you mean if you got a ?X", pat)

'What if you mean if you got a holiday'

In [128]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [150]:
def get_response(saying, response_rules):
    """
    根据规则做出回答
    :param saying:
    :param response_rules:
    :return:
    """
    for k, v in response_rules.items():
        succ, pat = match_sentence(k, saying)
        if succ:
            return subsitite(random.choice(v), pat)
    return "不好意思，我还没理解你说的是什么意思。"

In [175]:
a = ["1"]
a.extend(["a","b",["c","d"]])
a

['1', 'a', 'b', ['c', 'd']]

In [151]:
get_response("你好呀", rule_responses)

'很有趣'

In [152]:
get_response("你好呀小冰", rule_responses)

'我不太确定我很理解你说的能稍微详细解释一下吗'

In [153]:
get_response("你好小冰", rule_responses)

'很有趣'

In [154]:
get_response("小冰你好", rule_responses)

'很有趣'

In [160]:
get_response("小冰你好我想和你聊聊", rule_responses)

'你好呀'

In [186]:
get_response("我很讨厌上班", rule_responses)

'上班有什么不好呢'

# 问题
## 1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？
  
  优点就是程序非常简单，缺点也是非常简单，直接根据模板做出回答，句法稍微变化，程序就不能识别了

  另外匹配的效率可能有点低，把模式直接定义为正则表达式估计效率会提高一点,而且语法非常方便，比如中间允许空格，允许0次或多次匹配等，但是可能会难以维护

## 2. 什么是数据驱动？数据驱动在这个程序里如何体现？
  
  数据驱动是指问题的解决方案是由数据决定的，算法不变，因为数据的变化会驱动算法产生新的具体解法或方案的实例。例如这里，我们的解析模式以及匹配语句等代码都不用变，定义新的模式即可支持新的对话。

## 3. 数据驱动与 AI 的关系是什么？

 AI也是解决问题，由数据来产生具体解决方案的实例。比如分类问题，算法不变，由数据来拟合出具体的方程的参数，产生一个解。


